# Airbnb data analysis
### Questions:
>* What is the price range monthly in each region in Boston and Seattle?
>
>* What is the most vibe time in each region in Boston and Seattle?
>  
>* Can we predict the possible cost as per the corresponding holder's profiles (e.g., region, ratings, and month and day)?

# Load data

In [1]:
# data location
%ls ../../Datasets

Boston Airbnb Open Data/        Dataset of USED CARS.zip
Boston Airbnb Open Data.zip     Netflix_movie_and_TV_shows.csv
Car Sales.xlsx - car_data.csv   Netflix_movie_and_TV_shows.zip
Car sales report.zip            Seattle_Airbnb/
Dataset of USED CARS.csv        Seattle_Airbnb.zip


In [2]:
# set data location
data_dir = '../../Datasets/'
boston_dir = data_dir+"Boston Airbnb Open Data/"
seattle_dir = data_dir+'Seattle_Airbnb/'

In [3]:
import os
# all boston datasets and seattle datasets
bs_all,sa_all = [],[]
for root,dirs,files in os.walk(boston_dir):
    for file in files:
        bs_all.append(os.path.join(root,file))
for root,dirs,files in os.walk(seattle_dir):
    for file in files:
        sa_all.append(os.path.join(root,file))

> ## Load all datasets

In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)

In [5]:
# since both datasets contain 'reviews','listings', and 'calendar', create a dictionary key
dict_keys = ['reviews','listings','calendar']
# create dictionary of dataframes for both boston and seattle
dict_bs, dict_sa = {}, {}
for i,dict_key in enumerate(dict_keys):
    dict_bs[dict_key] = pd.read_csv(bs_all[i])
    dict_sa[dict_key] = pd.read_csv(sa_all[i])

> ## Wrangle data

> The data size is very large, directly merging will be too huge. Drop the non-essential columns and decrease the granuarity of the data.
>
> Focusing on the three questions shown above, NLP is not necessarily efficient in the case that the numerical ratings are given. Therefore, NLP remains to be optional for further analysis including keywords extraction and word vectorisation.

In [6]:
dict_sa['reviews'].sample(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
40372,4777832,51123160,2015-10-18,40688915,Shelley,This loft is amazing! The design inside is gre...
71057,447488,2068041,2012-08-22,2384415,Fernando,Spacious and confortable room. Located in one ...
21822,279063,15640883,2014-07-13,7198119,Daniele,Communication and reliability of Colen are per...
69230,1450262,33407357,2015-05-27,29341657,曉元,This house is so beautiful and in a quiet nice...
83250,2660384,24681316,2015-01-01,23398252,Vic,Perfect place to stay while in downtown Seattl...


In [7]:
dict_sa['listings'].sample(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
3543,6074312,https://www.airbnb.com/rooms/6074312,20160104002432,2016-01-04,Balcony room with Lake & Mt. Views,An upper floor room with queen bed and shared ...,Upper floor queen room with balcony access and...,An upper floor room with queen bed and shared ...,none,I am a Seattle native and I love my city. Capi...,Breakfast service is provided June through Sep...,Bus stops are located 1.5 blocks away. From th...,https://a1.muscache.com/ac/pictures/75688229/c...,https://a1.muscache.com/im/pictures/75688229/c...,https://a1.muscache.com/ac/pictures/75688229/c...,https://a1.muscache.com/ac/pictures/75688229/c...,12796487,https://www.airbnb.com/users/show/12796487,"Emma, Shannon, Anna & Rachel",2014-03-04,"Seattle, Washington, United States","We are a team of friendly, inspired hosts dedi...",within an hour,100%,100%,f,https://a0.muscache.com/ac/users/12796487/prof...,https://a0.muscache.com/ac/users/12796487/prof...,Stevens,6.0,6.0,"['email', 'phone', 'reviews', 'kba']",t,t,"East Denny Way, Seattle, WA 98122, United States",Stevens,Stevens,Capitol Hill,Seattle,WA,98122,Seattle,"Seattle, WA",US,United States,47.619203,-122.309389,t,Bed & Breakfast,Private room,2,1.0,1.0,1.0,Real Bed,{},NaN,$165.00,NaN,NaN,NaN,NaN,1,$0.00,2,28,2 weeks ago,t,30,60,90,153,2016-01-04,4,2015-08-03,2015-09-28,90.0,10.0,10.0,10.0,10.0,10.0,9.0,f,NaN,WASHINGTON,f,moderate,f,f,6,0.77


In [8]:
dict_sa['calendar'].sample(5)

,listing_id,date,available,price
1015841,9494415,2016-02-19,f,NaN
1133719,10332096,2016-02-02,t,$40.00
864154,4809486,2016-07-21,f,NaN
504643,4402880,2016-08-04,t,$225.00
208683,9347157,2016-09-28,t,$65.00


In [9]:
# save the dataframe for wrangling
ls_bs, ls_sa, cd_bs, cd_sa = dict_bs['listings'], dict_sa['listings'], dict_bs['calendar'], dict_sa['calendar']

In [10]:
# drop nans
ls_bs.dropna(how='all', axis=1, inplace=True)
ls_sa.dropna(how='all', axis=1, inplace=True)
ls_bs.dropna(how='all', axis=0, inplace=True)
ls_sa.dropna(how='all', axis=0, inplace=True)
# get the common columns for better comparision
ls_com_col = [col for col in ls_bs.columns if col in ls_sa.columns]
ls_bs, ls_sa = ls_bs[ls_com_col], ls_sa[ls_com_col]

In [11]:
ls_bs.sample(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
2307,4565470,https://www.airbnb.com/rooms/4565470,20160906204935,2016-09-07,Lux 2BR by Fenway w/WiFi,With its central location to Harvard Medical S...,At this luxurious property apartments are styl...,With its central location to Harvard Medical S...,none,"Characterized by cobblestone streets, clam cho...",Although we do not allow pets in our apartment...,"Most locals get around town by foot, giving Bo...",https://a0.muscache.com/im/pictures/79085999/c...,https://a0.muscache.com/im/pictures/79085999/c...,https://a0.muscache.com/im/pictures/79085999/c...,https://a0.muscache.com/im/pictures/79085999/c...,9419684,https://www.airbnb.com/users/show/9419684,Mike,2013-10-14,"New York, New York, United States",We have been providing vacation rental apartme...,within a few hours,96%,68%,f,https://a2.muscache.com/im/users/9419684/profi...,https://a2.muscache.com/im/users/9419684/profi...,West End,313,313,"['email', 'phone', 'manual_online', 'reviews',...",t,t,"Brookline Avenue, Boston, MA 02215, United States",Fenway/Kenmore,Fenway,Boston,MA,02215,Boston,"Boston, MA",US,United States,42.344724,-71.100097,t,Apartment,Entire home/apt,5,2.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$279.00,NaN,"$8,070.00",NaN,$100.00,1,$0.00,3,1125,4 days ago,30,60,86,86,2016-09-06,3,2015-01-04,2015-09-12,93.0,9.0,9.0,10.0,10.0,10.0,9.0,f,f,moderate,f,t,61,0.15
2296,11431331,https://www.airbnb.com/rooms/11431331,20160906204935,2016-09-07,"1085 Boylston St. ARTlab, 1 Bed Apt, Boston",Compass Furnished Apartments at 1085 Boylston ...,Compass Furnished Apartments at 1085 Boylston ...,Compass Furnished Apartments at 1085 Boylston ...,none,"Boston's Back Bay is an upscale residential, c...",For 30 or more day stays there will be a $1000...,- Walk to MBTA - Mass Pike,NaN,NaN,https://a2.muscache.com/im/pictures/102fee6c-e...,NaN,20857768,https://www.airbnb.com/users/show/20857768,Adam,2014-09-02,"Boston, Massachusetts, United States",My name is Adam and I am a Sales Representativ...,within a few hours,100%,67%,f,https://a2.muscache.com/im/users/20857768/prof...,https://a2.muscache.com/im/users/20857768/prof...,Downtown Crossing,15,15,"['email', 'phone', 'reviews', 'jumio']",t,t,"Boylston Street, Boston, MA 02215, United States",Fenway/Kenmore,Fenway,Boston,MA,02215,Boston,"Boston, MA",US,United States,42.347712,-71.090056,t,Apartment,Entire home/apt,3,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$289.00,NaN,NaN,NaN,NaN,1,$0.00,4,1125,today,19,49,79,79,2016-09-06,11,2016-03-17,2016-08-12,93.0,9.0,10.0,10.0,9.0,9.0,9.0,f,f,moderate,f,f,10,1.89


> To address the questions in this investigation, dataframe 'reviews' and NLP are not necessarily to be included.

> check the columns and select the essential columns
> 
> * 'id', 'neighbourhood_cleansed', 'price', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',	'review_scores_checkin', 'review_scores_communication',	'review_scores_location', 'review_scores_value', 'reviews_per_month'

In [12]:
select_ls_col = ['id', 'neighbourhood_cleansed', 'price', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

In [13]:
ls_bs, ls_sa = ls_bs[select_ls_col], ls_sa[select_ls_col]

In [14]:
ls_bs.sample(5)

,id,neighbourhood_cleansed,price,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
1066,8316068,South End,$110.00,96.0,10.0,10.0,10.0,10.0,10.0,10.0,2.31
715,14429876,North End,$60.00,93.0,9.0,7.0,9.0,9.0,9.0,9.0,3.00
1923,9464030,Beacon Hill,$150.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
857,13261006,Roxbury,$75.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2476,4916260,Fenway,$310.00,97.0,10.0,10.0,10.0,10.0,10.0,9.0,0.75


In [15]:
ls_sa.sample(5)

,id,neighbourhood_cleansed,price,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
2972,4023177,Loyal Heights,$119.00,96.0,10.0,9.0,10.0,10.0,10.0,9.0,4.52
2530,1450262,Bryant,$295.00,99.0,10.0,10.0,10.0,10.0,10.0,10.0,0.86
808,140331,Harrison/Denny-Blaine,$88.00,96.0,10.0,9.0,10.0,10.0,9.0,10.0,1.68
997,8424532,Leschi,$350.00,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.67
2482,8715320,Ravenna,$130.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
ls_bs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3585 entries, 0 to 3584
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           3585 non-null   int64  
 1   neighbourhood_cleansed       3585 non-null   object 
 2   price                        3585 non-null   object 
 3   review_scores_rating         2772 non-null   float64
 4   review_scores_accuracy       2762 non-null   float64
 5   review_scores_cleanliness    2767 non-null   float64
 6   review_scores_checkin        2765 non-null   float64
 7   review_scores_communication  2767 non-null   float64
 8   review_scores_location       2763 non-null   float64
 9   review_scores_value          2764 non-null   float64
 10  reviews_per_month            2829 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 308.2+ KB


> merge the listings and calendar for boston and seattle dataframes

In [20]:
# rename the price columns to avoid name clash
ls_bs.rename(columns={'price':'base_price'},inplace=True)

Index(['id', 'neighbourhood_cleansed', 'base_price', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'reviews_per_month'],
      dtype='object')

In [21]:
ls_sa.rename(columns={'price':'base_price'},inplace=True)

In [22]:
df_bs = ls_bs.merge(cd_bs, how='inner', left_on='id', right_on='listing_id')

,id,neighbourhood_cleansed,base_price,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,listing_id,date,available,price
1155516,10103725,South Boston,$250.00,93.0,10.0,8.0,10.0,10.0,10.0,10.0,1.27,10103725,2016-11-18,f,NaN
232258,8011545,North End,$79.00,98.0,10.0,10.0,10.0,10.0,10.0,10.0,3.96,8011545,2017-04-09,f,NaN
54582,210097,Jamaica Plain,$100.00,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.90,210097,2017-08-26,t,$100.00
417105,9853959,South End,$195.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9853959,2016-12-03,f,NaN
60773,27611,Jamaica Plain,$185.00,86.0,9.0,10.0,10.0,10.0,10.0,9.0,0.29,27611,2017-03-01,f,NaN


In [23]:
df_sa = ls_sa.merge(cd_sa, how='inner', left_on='id', right_on='listing_id')

In [24]:
# create an extra column for both
df_bs['city'], df_sa['city'] = 'Boston', 'Seattle'

In [26]:
df_bs.drop(columns='id', inplace=True)
df_sa.drop(columns='id', inplace=True)

In [35]:
# merge the two datasets
df = pd.concat([df_bs,df_sa], axis=0)

In [36]:
df.reset_index(inplace=True, drop=True)

In [39]:
df.isnull().sum()

neighbourhood_cleansed               0
base_price                           0
review_scores_rating            533265
review_scores_accuracy          540930
review_scores_cleanliness       537280
review_scores_checkin           539835
review_scores_communication     536550
review_scores_location          539470
review_scores_value             539470
reviews_per_month               505160
listing_id                           0
date                                 0
available                            0
price                          1124881
city                                 0
dtype: int64

In [40]:
df.shape

(2702460, 15)